<a href="https://cognitiveclass.ai"><img src = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/labs_v1/IDSNlogo.png" width = 400> </a>
​
<h1 align=center><font size = 5>Segmenting and Clustering Neighborhoods in Toronto</font></h1>
​

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2020.12.5  |       ha878542_0         137 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    geopy-2.1.0                |     pyhd3deb0d_0          64 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         235 KB

The following NEW packages will be INSTALLED:

  geographiclib      conda-forge/noarch::geographiclib-1.50-py_0
  geopy              conda-forge/noarch::geopy-2.1.0-pyhd3deb0d_0

The following packages will be SUPERSEDED by a higher-priority channel:

  ca-certificates    pkgs/main::ca-certificates-2021.1.19-~ --> conda-forge::ca-certificates-

#### First we load the data and prepare the dataframe

In [2]:
#Import data and clean it
data = pd.read_csv("TorontoNeighbours.csv")
Toronto_data=data[["Former city/borough","City-designated neighbourhood"]]
Toronto_data.columns=["Borough","Neighborhood"]
Toronto_data = Toronto_data.assign(Neighborhood=Toronto_data['Neighborhood'].str.split('-'))
Toronto_data = Toronto_data.explode('Neighborhood')
Toronto_data = Toronto_data.reset_index()
Toronto_data = Toronto_data.drop("index", 1)
Toronto_data.head()

,Borough,Neighborhood
0,Scarborough,Agincourt North
1,Scarborough,Agincourt South
2,Scarborough,Malvern West
3,Etobicoke,Alderwood
4,Old City of Toronto,Annex


Verify data

In [3]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(Toronto_data['Borough'].unique()),
        Toronto_data.shape[0]
    )
)

The dataframe has 6 boroughs and 206 neighborhoods.


#### Now we can check the coordinates of Toronto and its neighborhoods

First we set the geocode and create some useful functions

In [4]:
geolocator = Nominatim(user_agent="my_explorer")

#funtion that get the coordinates
def getCoordinatesOf(address):
    location = geolocator.geocode(address)
    if location == None:
        return None
    latitude = location.latitude
    longitude = location.longitude
    return([latitude, longitude])

Now we get the coordinates of Toronto

In [5]:
Toronto_coord = getCoordinatesOf('Toronto')
print('The geograpical coordinate of Toronto are {}, {}.'.format(Toronto_coord[0], Toronto_coord[1]))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


Create a map of Toronto with neighborhoods superimposed on top.

In [6]:
# create map of New York using latitude and longitude values
map_Toronto = folium.Map(location=Toronto_coord, zoom_start=10)

# add markers to map
for borough, neighborhood in zip(Toronto_data['Borough'], Toronto_data['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    coord = getCoordinatesOf(neighborhood)
    if coord != None:
        folium.CircleMarker(
            coord,
            radius=5,
            popup=label,
            color='blue',
            fill=True,
            fill_color='#3186cc',
            fill_opacity=0.7,
            parse_html=False).add_to(map_Toronto)  
    
map_Toronto

Let's see which Toronto districts have the most neighborhoods

In [11]:
for borough in Toronto_data['Borough'].unique():
    neighs = Toronto_data[Toronto_data['Borough'] == borough].shape[0]
    print("{}: {}".format(borough, neighs))

Scarborough: 30
Etobicoke: 36
Old City of Toronto: 63
North York: 49
York: 16
East York: 12


Old City of Toronto is the district with more neighbours, let's see the dataframe

In [12]:
OCT_data = Toronto_data[Toronto_data['Borough'] == 'Old City of Toronto']
OCT_data.head()

,Borough,Neighborhood
4,Old City of Toronto,Annex
8,Old City of Toronto,Bay Street Corridor
20,Old City of Toronto,Blake
21,Old City of Toronto,Jones
30,Old City of Toronto,Cabbagetown


And create the map

In [9]:
OCT_coord = getCoordinatesOf('Old City of Toronto')
OCT_map = folium.Map(location=OCT_coord, zoom_start=10)

# add markers to map
for neighborhood in zip(OCT_data['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    coord = getCoordinatesOf(neighborhood)
    if coord != None:
        folium.CircleMarker(
            coord,
            radius=5,
            popup=label,
            color='blue',
            fill=True,
            fill_color='#3186cc',
            fill_opacity=0.7,
            parse_html=False).add_to(OCT_map)  
OCT_map

Let's get the top 100 venues that are in Annex within a radius of 500 meters.

We start by defining the GET request URL

In [10]:
Annex_coord = getCoordinatesOf('Annex')
Annex_coord
# type your answer here
CLIENT_ID = '4JW2V1AXWJXREM12VKKJCM4B50FZ1I4TYRS2URMC1WMZRTNM' # your Foursquare ID
CLIENT_SECRET = 'MIJZAJNLVUZ0GB1DWIEPGAK4XXJVQFFZFOUT2EN1DHA4E2D4' # your Foursquare Secret
ACCESS_TOKEN = 'VP2X2PUFBQYAVKTTZUEIUCB3QYSNE5BI3ZOZSGOXU2XQZSTE' # your FourSquare Access Token
VERSION = '20180604'
LIMIT = 30
radius = 500
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    Annex_coord[0], 
    Annex_coord[1], 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=4JW2V1AXWJXREM12VKKJCM4B50FZ1I4TYRS2URMC1WMZRTNM&client_secret=MIJZAJNLVUZ0GB1DWIEPGAK4XXJVQFFZFOUT2EN1DHA4E2D4&v=20180604&ll=43.6703377,-79.407117&radius=500&limit=30'

Send the GET request and examine the results

In [13]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '6071b9e51937a72420d00e2a'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'The Annex',
  'headerFullLocation': 'The Annex, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 37,
  'suggestedBounds': {'ne': {'lat': 43.6748377045, 'lng': -79.40090733739737},
   'sw': {'lat': 43.665837695499995, 'lng': -79.41332666260263}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4c0c36c0a1b32d7f97439cf0',
       'name': 'Jean Sibelius Square',
       'location': {'address': 'Wells St and Kendal Ave.',
        'lat': 43.67142628683921,
        'lng': -79.40883090368688,
        'labeledLatLngs': [{'label': 'display',
          'lat'

From the Foursquare lab in the previous module, we know that all the information is in the _items_ key. Before we proceed, let's borrow the **get_category_type** function from the Foursquare lab.

In [14]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

and pass the cleaned JSON data in a pandas dataframe 

In [15]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues = nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  This is separate from the ipykernel package so we can avoid doing imports until


,name,categories,lat,lng
0,Jean Sibelius Square,Park,43.671426,-79.408831
1,Roti Cuisine of India,Indian Restaurant,43.674618,-79.408249
2,Fuwa Fuwa Japanese Pancakes,Pastry Shop,43.665880,-79.407840
3,COBS Bread,Bakery,43.666222,-79.406066
4,The Original Gyro Grill,Greek Restaurant,43.666621,-79.405544


Let's create a function to repeat the same process to all the neighborhoods in Old City of Toronto 

In [16]:
def getNearbyVenues(neighs, radius=500):
    
    venues_list=[]
    for neigh in neighs:
        print(neigh)
        coord = getCoordinatesOf(neigh)
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            coord[0], 
            coord[1], 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            neigh, 
            coord[0], 
            coord[1], 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

and call it

In [17]:
OCT_venues = getNearbyVenues(neighs=OCT_data['Neighborhood'])

Annex
Bay Street Corridor
Blake
Jones
Cabbagetown
South St. James Town
Casa Loma
Church
Yonge Corridor
Corso Italia
Davenport
Danforth Village 
 Toronto
Dovercourt
Wallace Emerson
Junction
Dufferin Grove
East End
Danforth
Forest Hill North
Forest Hill South
Greenwood
Coxwell
High Park North
High Park
Swansea
Junction Area
Kensington
Chinatown
Lawrence Park North
Lawrence Park South
Little Portugal
Moss Park
Mount Pleasant East
Mount Pleasant West
Niagara
North Riverdale
North St. James Town
Palmerston
Little Italy
Playter Estates
Danforth
Regent Park
Roncesvalles
Rosedale
Moore Park
Runnymede
Bloor West Village
South Riverdale
South Parkdale
The Beaches
Trinity
Bellwoods
University
Waterfront Communities
The Island
Weston
Pellam Park
Woodbine Corridor
Wychwood
Yonge and Eglinton
Yonge
St.Clair


Let'see the dataframe

In [18]:
print(OCT_venues.shape)
OCT_venues.head()

(925, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Annex,43.670338,-79.407117,Jean Sibelius Square,43.671426,-79.408831,Park
1,Annex,43.670338,-79.407117,Roti Cuisine of India,43.674618,-79.408249,Indian Restaurant
2,Annex,43.670338,-79.407117,Fuwa Fuwa Japanese Pancakes,43.665880,-79.407840,Pastry Shop
3,Annex,43.670338,-79.407117,COBS Bread,43.666222,-79.406066,Bakery
4,Annex,43.670338,-79.407117,The Original Gyro Grill,43.666621,-79.405544,Greek Restaurant


Let's check how many venues were returned for each neighborhood

In [19]:
pd.DataFrame(OCT_venues.groupby('Neighborhood')['Venue'].count())

,Venue
Neighborhood,
Toronto,30
Annex,30
Bay Street Corridor,30
Bellwoods,30
Bloor West Village,30
Cabbagetown,30
Casa Loma,22
Chinatown,30
Church,1


Let's find out how many unique categories can be curated from all the returned venues

In [20]:
print('There are {} uniques categories.'.format(len(OCT_venues['Venue Category'].unique())))

There are 204 uniques categories.


Now analyze each neighborhood

In [21]:
#one hot encoding
OCT_onehot = pd.get_dummies(OCT_venues[['Venue Category']])
OCT_onehot.columns = [col.split("_")[-1] for col in OCT_onehot.columns]

# add neighborhood column back to dataframe
OCT_onehot['Neighborhood'] = OCT_venues['Neighborhood']

# move neighborhood column to the first column
index=OCT_onehot.columns.get_loc('Neighborhood')
fixed_columns = [OCT_onehot.columns[index]] + list(OCT_onehot.columns[:index]) + list(OCT_onehot.columns[index+1:])
OCT_onehot = OCT_onehot[fixed_columns]

OCT_onehot.head()


,Neighborhood,ATM,Accessories Store,American Restaurant,Animal Shelter,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,BBQ Joint,Bakery,Bank,Bar,Baseball Field,Basketball Court,Beach,Beer Bar,Beer Store,Bike Rental / Bike Share,Bike Shop,Bistro,Bookstore,Botanical Garden,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridge,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Stop,Butcher,Café,Candy Store,Caribbean Restaurant,Castle,Chinese Restaurant,Chocolate Shop,Clothing Store,Cocktail Bar,Coffee Shop,Comedy Club,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Cricket Ground,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Donut Shop,Dry Cleaner,Dumpling Restaurant,Electronics Store,English Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Food,Food & Drink Shop,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Garden,Gas Station,Gastropub,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hotel,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kids Store,Korean Restaurant,Latin American Restaurant,Library,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts School,Massage Studio,Mattress Store,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Movie Theater,Multiplex,Museum,Music Venue,Nail Salon,Opera House,Optical Shop,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Post Office,Print Shop,Pub,Racetrack,Ramen Restaurant,Recording Studio,Residential Building (Apartment / Condo),Restaurant,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,School,Science Museum,Seafood Restaurant,Shopping Mall,Shopping Plaza,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Supermarket,Sushi Restaurant,Taco Place,Taiwanese Restaurant,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Park Ride / Attraction,Toy / Game Store,Trail,Tram Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Annex,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Annex,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Annex,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

and verify data size

In [22]:
OCT_onehot.shape

(925, 204)

Now we group the neighborhoods by the average of each category

In [23]:
OCT_grouped=OCT_onehot.groupby('Neighborhood').mean().reset_index()
OCT_grouped

,Neighborhood,ATM,Accessories Store,American Restaurant,Animal Shelter,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,BBQ Joint,Bakery,Bank,Bar,Baseball Field,Basketball Court,Beach,Beer Bar,Beer Store,Bike Rental / Bike Share,Bike Shop,Bistro,Bookstore,Botanical Garden,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridge,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Stop,Butcher,Café,Candy Store,Caribbean Restaurant,Castle,Chinese Restaurant,Chocolate Shop,Clothing Store,Cocktail Bar,Coffee Shop,Comedy Club,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Cricket Ground,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Donut Shop,Dry Cleaner,Dumpling Restaurant,Electronics Store,English Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Food,Food & Drink Shop,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Garden,Gas Station,Gastropub,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hotel,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kids Store,Korean Restaurant,Latin American Restaurant,Library,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts School,Massage Studio,Mattress Store,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Movie Theater,Multiplex,Museum,Music Venue,Nail Salon,Opera House,Optical Shop,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Post Office,Print Shop,Pub,Racetrack,Ramen Restaurant,Recording Studio,Residential Building (Apartment / Condo),Restaurant,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,School,Science Museum,Seafood Restaurant,Shopping Mall,Shopping Plaza,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Supermarket,Sushi Restaurant,Taco Place,Taiwanese Restaurant,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Park Ride / Attraction,Toy / Game Store,Trail,Tram Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Toronto,0.000000,0.000000,0.033333,0.000000,0.000000,0.000000,0.033333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.033333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.033333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.033333,0.000000,0.066667,0.000000,0.033333,0.000000,0.000000,0.033333,0.000000,0.066667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.033333,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.066667,0.000000,0.000000,0.000000,0.000000,0.000000,0.033333,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.033333,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00

and verify size

In [24]:
OCT_grouped.shape

(45, 204)

Let's print each neighborhood along with the top 5 most common venues

In [25]:
num_top_venues = 5

for hood in OCT_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = OCT_grouped[OCT_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

---- Toronto----
               venue  freq
0  Electronics Store  0.07
1     Clothing Store  0.07
2              Plaza  0.07
3            Theater  0.07
4     Cosmetics Shop  0.07


----Annex----
                 venue  freq
0          Pizza Place  0.10
1                 Park  0.07
2               Bistro  0.07
3      Thai Restaurant  0.07
4  Fried Chicken Joint  0.03


----Bay Street Corridor----
                venue  freq
0  Italian Restaurant  0.10
1               Hotel  0.07
2        Gourmet Shop  0.07
3   French Restaurant  0.07
4            Boutique  0.07


----Bellwoods----
          venue  freq
0   Yoga Studio  0.07
1        Bakery  0.07
2  Cocktail Bar  0.07
3     Pet Store  0.07
4   Pizza Place  0.07


----Bloor West Village----
         venue  freq
0         Café  0.10
1        Diner  0.07
2  Pizza Place  0.07
3  Coffee Shop  0.07
4          Pub  0.07


----Cabbagetown----
                 venue  freq
0                 Café  0.10
1           Restaurant  0.10
2  Japanese Resta

Let's put that into a pandas dataframe

First, let's write a function to sort the venues in descending order

In [26]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 10 venues for each neighborhood

In [27]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
OCT_venues_sorted = pd.DataFrame(columns=columns)
OCT_venues_sorted['Neighborhood'] = OCT_grouped['Neighborhood']

for ind in np.arange(OCT_grouped.shape[0]):
    OCT_venues_sorted.iloc[ind, 1:] = return_most_common_venues(OCT_grouped.iloc[ind, :], num_top_venues)

OCT_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Toronto,Cosmetics Shop,Plaza,Clothing Store,Theater,Electronics Store,Poke Place,Restaurant,Shopping Mall,Seafood Restaurant,Bubble Tea Shop
1,Annex,Pizza Place,Park,Thai Restaurant,Bistro,Grocery Store,Burrito Place,Bookstore,Fast Food Restaurant,Falafel Restaurant,Middle Eastern Restaurant
2,Bay Street Corridor,Italian Restaurant,Hotel,Boutique,French Restaurant,Spa,Gourmet Shop,Gastropub,Indian Restaurant,Japanese Restaurant,Furniture / Home Store
3,Bellwoods,Yoga Studio,Café,Bakery,Cocktail Bar,Pizza Place,Pet Store,Park,Butcher,Restaurant,Paper / Office Supplies Store
4,Bloor West Village,Café,Pub,Pizza Place,Diner,Coffee Shop,Sushi Restaurant,Bar,Sandwich Place,Fish & Chips Shop,Smoothie Shop


Now we have to Cluster Neighborhoods

Run k-means to cluster the neighborhood into 5 clusters

In [28]:
# set number of clusters
kclusters = 5

OCT_grouped_clustering = OCT_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(OCT_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_

array([0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 3, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 4, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0], dtype=int32)

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood

In [ ]:
# add clustering labels
OCT_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

OCT_merged = OCT_data

# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
OCT_merged = OCT_merged.join(OCT_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

OCT_merged# check the last columns!

,Borough,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,Old City of Toronto,Annex,0.0,Pizza Place,Park,Thai Restaurant,Bistro,Grocery Store,Burrito Place,Bookstore,Fast Food Restaurant,Falafel Restaurant,Middle Eastern Restaurant
8,Old City of Toronto,Bay Street Corridor,0.0,Italian Restaurant,Hotel,Boutique,French Restaurant,Spa,Gourmet Shop,Gastropub,Indian Restaurant,Japanese Restaurant,Furniture / Home Store
20,Old City of Toronto,Blake,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21,Old City of Toronto,Jones,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
30,Old City of Toronto,Cabbagetown,0.0,Restaurant,Café,Bakery,Gastropub,Italian Restaurant,Japanese Restaurant,Diner,Thai Restaurant,Pub,Beer Store
31,Old City of Toronto,South St. James Town,0.0,Coffee Shop,Pizza Place,Café,Grocery Store,Hotel,Bistro,Library,Market,Food & Drink Shop,Filipino Restaurant
34,Old City of Toronto,Casa Loma,0.0,Coffee Shop,Sandwich Place,Café,History Museum,Castle,Museum,Steakhouse,Theater,Donut Shop,Middle Eastern Restaurant
36,Old City of Toronto,Church,1.0,Racetrack,Yoga Studio,Dry Cleaner,Filipino Restaurant,Field,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant
37,Old City of Toronto,Yonge Corridor,0.0,Hotel,Indian Restaurant,Japanese Restaurant,Gourmet Shop,Shopping Mall,Salad Place,Restaurant,Pizza Place,Middle Eastern Restaurant,Mediterranean Restaurant
42,Old City of Toronto,Corso Italia,0.0,Italian Restaurant,Breakfast Spot,Sushi Restaurant,Coffee Shop,Thai Restaurant,Brazilian Restaurant,Mexican Restaurant,Pet Store,Lounge,Sandwich Place
